Task 3: Store Cleaned Data in Oracle

<li>Creates database schema
<li>Loads data into Oracle
<li>Exports SQL dump

In [4]:
# import necessary libraries
import oracledb
import pandas as pd
import logging
import os
from datetime import datetime

In [ ]:
pip install oracledb

  Using cached cx_Oracle-8.3.0.tar.gz (363 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build cx_Oracle
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for cx_Oracle (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      <string>:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
      C:\Users\you\AppData\Local\Temp\pip-build-env-ceqs7yt8\overlay\Lib\site-packages\setuptools\config\expand.py:126: SetuptoolsWarning: File 'C:\\Users\\you\\AppData\\Local\\Temp\\pip-install-3b6f8huu\\cx-oracle_6310d036522a4c3aad976807b37ebffc\\README.md' cannot be found
        return '\n'.join(
      C:\Users\you\AppData\Local\Temp\pip-build-env-ceqs7yt8\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              **********************************************************************

In [5]:
# Set up logging
logging.basicConfig(filename='database.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:

# Oracle connection details
ORACLE_USER = "bank_reviews"
ORACLE_PASSWORD = "shalom2611"  # In production, use environment variables
ORACLE_DSN = "localhost:1521/XEPDB1"  # Update with your Oracle connection string

In [ ]:
def create_database_schema():
    """Create the necessary tables in Oracle database"""
    logging.info("Setting up database schema...")
    
    try:
        connection = cx_Oracle.connect(ORACLE_USER, ORACLE_PASSWORD, ORACLE_DSN)
        cursor = connection.cursor()
        
        # Create banks table
        cursor.execute("""
        CREATE TABLE banks (
            bank_id NUMBER GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
            bank_name VARCHAR2(100) UNIQUE NOT NULL,
            app_id VARCHAR2(100) UNIQUE NOT NULL,
            created_date DATE DEFAULT SYSDATE
        )
        """)
        
        # Create reviews table
        cursor.execute("""
        CREATE TABLE reviews (
            review_id VARCHAR2(100) PRIMARY KEY,
            bank_id NUMBER NOT NULL,
            review_text CLOB,
            rating NUMBER(1),
            review_date DATE,
            sentiment_label VARCHAR2(20),
            sentiment_score NUMBER(5,4),
            identified_themes VARCHAR2(200),
            source VARCHAR2(50),
            created_date DATE DEFAULT SYSDATE,
            CONSTRAINT fk_bank FOREIGN KEY (bank_id) REFERENCES banks(bank_id)
        )
        """)
        
        # Insert bank data
        cursor.executemany("""
        INSERT INTO banks (bank_name, app_id) VALUES (:1, :2)
        """, [
            ('Dashen Bank', 'com.dashen.dashensuperapp'),
            ('Commercial Bank of Ethiopia', 'com.cbe.dfc.asc.android'),
            ('Bank of Abyssinia', 'com.boa.android')
        ])
        
        connection.commit()
        logging.info("Database schema created successfully")
        
    except cx_Oracle.Error as error:
        logging.error(f"Database error: {error}")
    finally:
        if 'connection' in locals():
            connection.close()

In [ ]:
def load_data_to_oracle(csv_path):
    """Load the analyzed data into Oracle database"""
    logging.info(f"Loading data from {csv_path} to Oracle...")
    
    try:
        # Load data from CSV
        df = pd.read_csv(csv_path)
        logging.info(f"Loaded {len(df)} records from CSV")
        
        # Connect to Oracle
        connection = cx_Oracle.connect(ORACLE_USER, ORACLE_PASSWORD, ORACLE_DSN)
        cursor = connection.cursor()
        
        # Get bank IDs
        cursor.execute("SELECT bank_id, bank_name FROM banks")
        bank_mapping = {name: id for id, name in cursor.fetchall()}
        
        # Prepare data for insertion
        insert_data = []
        for _, row in df.iterrows():
            bank_name = row['bank_name']
            if bank_name not in bank_mapping:
                logging.warning(f"Bank '{bank_name}' not found in database")
                continue
                
            bank_id = bank_mapping[bank_name]
            
            # Convert date string to datetime object
            try:
                review_date = datetime.strptime(row['date'], '%Y-%m-%d')
            except:
                review_date = None
            
            insert_data.append((
                row['review_id'],
                bank_id,
                row['review_text'] if pd.notna(row['review_text']) else "",
                int(row['rating']) if pd.notna(row['rating']) else None,
                review_date,
                row['sentiment_label'] if 'sentiment_label' in row and pd.notna(row['sentiment_label']) else None,
                float(row['sentiment_score']) if 'sentiment_score' in row and pd.notna(row['sentiment_score']) else None,
                row['identified_themes'] if 'identified_themes' in row and pd.notna(row['identified_themes']) else None,
                row['source'] if pd.notna(row['source']) else None
            ))
        
        # Insert data in batches
        batch_size = 100
        for i in range(0, len(insert_data), batch_size):
            batch = insert_data[i:i+batch_size]
            cursor.executemany("""
            INSERT INTO reviews (
                review_id, bank_id, review_text, rating, review_date,
                sentiment_label, sentiment_score, identified_themes, source
            ) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9)
            """, batch)
            connection.commit()
            logging.info(f"Inserted batch {i//batch_size + 1}/{(len(insert_data)-1)//batch_size + 1}")
        
        logging.info(f"Successfully loaded {len(insert_data)} records to Oracle")
        
    except Exception as e:
        logging.error(f"Error loading data to Oracle: {e}")
    finally:
        if 'connection' in locals():
            connection.close()

def export_sql_dump():
    """Export a SQL dump of the database"""
    logging.info("Exporting SQL dump...")
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    dump_file = f"bank_reviews_dump_{timestamp}.sql"
    
    try:
        # Use Oracle's exp utility for export
        os.system(f"exp {ORACLE_USER}/{ORACLE_PASSWORD}@{ORACLE_DSN} file={dump_file} log=export.log")
        logging.info(f"SQL dump exported to {dump_file}")
        return dump_file
    except Exception as e:
        logging.error(f"Error exporting SQL dump: {e}")
        return None

In [ ]:
def run_database_setup(csv_path):
    """Run the complete database setup and data loading process"""
    try:
        # Initialize Oracle client
        cx_Oracle.init_oracle_client()
        
        # Create schema
        create_database_schema()
        
        # Load data
        load_data_to_oracle(csv_path)
        
        # Export SQL dump
        export_sql_dump()
        
        logging.info("Database setup complete")
        
    except Exception as e:
        logging.error(f"Database setup failed: {e}")

if __name__ == "__main__":
    # Replace with your analyzed data file
    input_file = "all_banks_reviews_clean_20250611_000000_analyzed.csv"  # Update with your actual filename
    run_database_setup(input_file)